# Workshop Notebook 3: Observability Part 1 - Validation Rules

In the previous notebooks you uploaded the models and artifacts, then deployed the models to production through provisioning workspaces and pipelines. Now you're ready to put your feet up! But to keep your models operational, your work's not done once the model is in production. You must continue to monitor the behavior and performance of the model to insure that the model provides value to the business.

In this notebook, you will learn about adding validation rules to pipelines.

## Preliminaries

In the blocks below we will preload some required libraries.

In [1]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework


from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import datetime
import time


## Login to Wallaroo

Retrieve the previous workspace, model versions, and pipelines used in the previous notebook.

In [2]:
## blank space to log in 

wl = wallaroo.Client()

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-05"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'house-price-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'houseprice-estimator'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-workspace-john-05', 'id': 11, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-04-22T20:33:29.332164+00:00', 'models': [{'name': 'house-price-prime', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 47, 17, 816549, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 34, 37, 434083, tzinfo=tzutc())}, {'name': 'house-price-rf-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc())}, {'name': 'house-price-gbr-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc())}], 'pipelines': [{'name': 'houseprice-estimator', 'create_time': datetime.datetime(2024, 4, 22, 20, 34, 49, 55

Name,house-price-prime
Version,be93bbfc-4f0c-444d-aa89-5f15dec01d83
File Name,xgb_model.onnx
SHA,31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-22-Apr 20:47:17


name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-22 21:02:57.163023+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-rf-model
published,False


## Deploy the Pipeline

Add the model version as a pipeline step to our pipeline, and deploy the pipeline.  You may want to check the pipeline steps to verify that the right model version is set for the pipeline step.

In [3]:
## blank space to get your pipeline and run a small batch of data through it to see the range of predictions

pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

multiple_result = pipeline.infer_from_file('../data/test_data.df.json')
display(multiple_result)


,time,in.tensor,out.variable,anomaly.count
0,2024-04-22 21:04:29.814,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0
1,2024-04-22 21:04:29.814,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[732883.5],0
2,2024-04-22 21:04:29.814,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[419508.84],0
3,2024-04-22 21:04:29.814,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[634028.75],0
4,2024-04-22 21:04:29.814,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[427209.47],0
...,...,...,...,...
3995,2024-04-22 21:04:29.814,"[4.0, 2.25, 2620.0, 98881.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1820.0, 800.0, 47.4662, -122.453, 1728.0, 95832.0, 63.0, 0.0, 0.0]",[436151.13],0
3996,2024-04-22 21:04:29.814,"[3.0, 2.5, 2244.0, 4079.0, 2.0, 0.0, 0.0, 3.0, 7.0, 2244.0, 0.0, 47.2606, -122.254, 2077.0, 4078.0, 3.0, 0.0, 0.0]",[284810.7],0
3997,2024-04-22 21:04:29.814,"[3.0, 1.75, 1490.0, 5000.0, 1.0, 0.0, 1.0, 3.0, 8.0, 1250.0, 240.0, 47.5257, -122.392, 1980.0, 5000.0, 61.0, 0.0, 0.0]",[575571.44],0
3998,2024-04-22 21:04:29.814,"[4.0, 2.5, 2740.0, 5700.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2740.0, 0.0, 47.3535, -122.026, 3010.0, 5281.0, 8.0, 0.0, 0.0]",[432262.38],0


## Model Validation Rules

A simple way to try to keep your model's behavior up to snuff is to make sure that it receives inputs that it expects, and that its output is something that downstream systems can handle. This can entail specifying rules that document what you expect, and either enforcing these rules (by refusing to make a prediction), or at least logging an alert that the expectations described by your validation rules have been violated. As the developer of the model, the data scientist (along with relevant subject matter experts) will often be the person in the best position to specify appropriate validation rules.

In our house price prediction example, suppose you know that house prices in your market are typically in the range $750,000 to $1.5M dollars. Then you might want to set validation rules on your model pipeline to specify that you expect the model's predictions to also be in that range. Then, if the model predicts a value outside that range, the pipeline will log that one of the validation checks has failed; this allows you to investigate that instance further.

Wallaroo provides **validations** to detect anomalous data from inference inputs and outputs.  Validations are added to a Wallaroo pipeline with the `wallaroo.pipeline.add_validations` method.

Adding validations takes the format:

```python
pipeline.add_validations(
    validation_name_01 = polars.col(in|out.{column_name}) EXPRESSION,
    validation_name_02 = polars.col(in|out.{column_name}) EXPRESSION
    ...{additional rules}
)
```

* `validation_name`: The user provided name of the validation.  The names must match Python variable naming requirements.
  * **IMPORTANT NOTE**: Using the name `count` as a validation name **returns a warning**.  Any validation rules named `count` are dropped upon request and an warning returned.
* `polars.col(in|out.{column_name})`: Specifies the **input** or **output** for a specific field aka "column" in an inference result.  Wallaroo inference requests are in the format `in.{field_name}` for **inputs**, and `out.{field_name}` for **outputs**.
  * More than one field can be selected, as long as they follow the rules of the [polars 0.18 Expressions library](https://docs.pola.rs/docs/python/version/0.18/reference/expressions/index.html).
* `EXPRESSION`:  The expression to validate. When the expression returns **True**, that indicates an anomaly detected.

The [`polars` library version 0.18.5](https://docs.pola.rs/docs/python/version/0.18/index.html) is used to create the validation rule.  This is installed by default with the Wallaroo SDK.  This provides a powerful range of comparisons to organizations tracking anomalous data from their ML models.

When validations are added to a pipeline, inference request outputs return the following fields:

| Field | Type | Description |
|---|---|---|
| **anomaly.count** | **Integer** | The total of all validations that returned **True**. |
| **anomaly.{validation name}** | **Bool** | The output of the validation `{validation_name}`. |

When validation returns `True`, **an anomaly is detected**.

For example, adding the validation `fraud` to the following pipeline returns `anomaly.count` of `1` when the validation `fraud` returns `True`.  The validation `fraud` returns `True` when the **output** field **dense_1** at index **0** is greater than 0.9.

```python
sample_pipeline = wallaroo.client.build_pipeline("sample-pipeline")
sample_pipeline.add_model_step(model)

# add the validation
sample_pipeline.add_validations(
    fraud=pl.col("out.dense_1").list.get(0) > 0.9,
    )

# deploy the pipeline
sample_pipeline.deploy()

# sample inference
display(sample_pipeline.infer_from_file("dev_high_fraud.json", data_format='pandas-records'))
```

|&nbsp;|time|in.tensor|out.dense_1|anomaly.count|anomaly.fraud|
|---|---|---|---|---|---|
|0|2024-02-02 16:05:42.152|[1.0678324729, 18.1555563975, -1.6589551058, 5...]|[0.981199]|1|True|

### Detecting Anomalies from Inference Request Results

When an inference request is submitted to a Wallaroo pipeline with validations, the following fields are output:

| Field | Type | Description |
|---|---|---|
| **anomaly.count** | **Integer** | The total of all validations that returned **True**. |
| **anomaly.{validation name}** | **Bool** | The output of each pipeline validation `{validation_name}`. |

For example, adding the validation `fraud` to the following pipeline returns `anomaly.count` of `1` when the validation `fraud` returns `True`.

```python
sample_pipeline = wallaroo.client.build_pipeline("sample-pipeline")
sample_pipeline.add_model_step(model)

# add the validation
sample_pipeline.add_validations(
    fraud=pl.col("out.dense_1").list.get(0) > 0.9,
    )

# deploy the pipeline
sample_pipeline.deploy()

# sample inference
display(sample_pipeline.infer_from_file("dev_high_fraud.json", data_format='pandas-records'))
```

|&nbsp;|time|in.tensor|out.dense_1|anomaly.count|anomaly.fraud|
|---|---|---|---|---|---|
|0|2024-02-02 16:05:42.152|[1.0678324729, 18.1555563975, -1.6589551058, 5...]|[0.981199]|1|True|

### Model Validation Rules Exercise

Add some simple validation rules to the model pipeline that you created in a previous exercise.

* Add an upper bound or a lower bound to the model predictions.
* Try to create predictions that fall both in and out of the specified range.
* Look through the logs to find anomalies.

**HINT 1**: since the purpose of this exercise is try out validation rules, it might be a good idea to take a small data set and make predictions on that data set first, *then* set the validation rules based on those predictions, so that you can see the check failures trigger.

Here's an example:

```python
import polars as pl

sample_pipeline = sample_pipeline.add_validations(
    too_high=pl.col("out.variable").list.get(0) > 1000000.0
)
```

In [ ]:
# blank space to set a validation rule on the pipeline and check if it triggers as expected

import polars as pl

pipeline = pipeline.add_validations(
    too_high=pl.col("out.variable").list.get(0) > 1000000.0
)


In [5]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

multiple_result = pipeline.infer_from_file('../data/test_data.df.json')

# show the first to results
display(multiple_result.head(20))

# show only the results that triggered an anomaly
multiple_result.loc[multiple_result['anomaly.count'] > 0]

,time,in.tensor,out.variable,anomaly.count,anomaly.too_high
0,2024-04-22 21:06:25.675,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0,False
1,2024-04-22 21:06:25.675,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[732883.5],0,False
2,2024-04-22 21:06:25.675,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[419508.84],0,False
3,2024-04-22 21:06:25.675,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[634028.75],0,False
4,2024-04-22 21:06:25.675,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[427209.47],0,False
5,2024-04-22 21:06:25.675,"[3.0, 2.0, 2140.0, 4923.0, 1.0, 0.0, 0.0, 4.0, 8.0, 1070.0, 1070.0, 47.6902, -122.339, 1470.0, 4923.0, 86.0, 0.0, 0.0]",[615501.9],0,False
6,2024-04-22 21:06:25.675,"[4.0, 3.5, 3590.0, 5334.0, 2.0, 0.0, 2.0, 3.0, 9.0, 3140.0, 450.0, 47.6763, -122.267, 2100.0, 6250.0, 9.0, 0.0, 0.0]",[1139732.4],1,True
7,2024-04-22 21:06:25.675,"[3.0, 2.0, 1280.0, 960.0, 2.0, 0.0, 0.0, 3.0, 9.0, 1040.0, 240.0, 47.602, -122.311, 1280.0, 1173.0, 0.0, 0.0, 0.0]",[498328.88],0,False
8,2024-04-22 21:06:25.675,"[4.0, 2.5, 2820.0, 15000.0, 2.0, 0.0, 0.0, 4.0, 9.0, 2820.0, 0.0, 47.7255, -122.101, 2440.0, 15000.0, 29.0, 0.0, 0.0]",[722664.4],0,False
9,2024-04-22 21:06:25.675,"[3.0, 2.25, 1790.0, 11393.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1790.0, 0.0, 47.6297, -122.099, 2290.0, 11894.0, 36.0, 0.0, 0.0]",[525746.44],0,False


,time,in.tensor,out.variable,anomaly.count,anomaly.too_high
6,2024-04-22 21:06:25.675,"[4.0, 3.5, 3590.0, 5334.0, 2.0, 0.0, 2.0, 3.0, 9.0, 3140.0, 450.0, 47.6763, -122.267, 2100.0, 6250.0, 9.0, 0.0, 0.0]",[1139732.4],1,True
30,2024-04-22 21:06:25.675,"[4.0, 3.0, 3710.0, 20000.0, 2.0, 0.0, 2.0, 5.0, 10.0, 2760.0, 950.0, 47.6696, -122.261, 3970.0, 20000.0, 79.0, 0.0, 0.0]",[2176827.0],1,True
35,2024-04-22 21:06:25.675,"[4.0, 2.5, 2440.0, 3600.0, 2.5, 0.0, 0.0, 4.0, 8.0, 2440.0, 0.0, 47.6298, -122.362, 2440.0, 5440.0, 112.0, 0.0, 0.0]",[1040167.6],1,True
40,2024-04-22 21:06:25.675,"[4.0, 4.5, 5120.0, 41327.0, 2.0, 0.0, 0.0, 3.0, 10.0, 3290.0, 1830.0, 47.7009, -122.059, 3360.0, 82764.0, 6.0, 0.0, 0.0]",[1092150.5],1,True
51,2024-04-22 21:06:25.675,"[4.0, 2.5, 3430.0, 35120.0, 2.0, 0.0, 0.0, 3.0, 10.0, 3430.0, 0.0, 47.6484, -122.182, 3920.0, 35230.0, 31.0, 0.0, 0.0]",[1035137.1],1,True
...,...,...,...,...,...
3885,2024-04-22 21:06:25.675,"[4.0, 1.5, 3120.0, 7680.0, 1.0, 0.0, 3.0, 3.0, 8.0, 1660.0, 1460.0, 47.646, -122.39, 2900.0, 7680.0, 58.0, 0.0, 0.0]",[1111801.4],1,True
3910,2024-04-22 21:06:25.675,"[3.0, 2.5, 2280.0, 5000.0, 2.0, 0.0, 3.0, 4.0, 9.0, 2280.0, 0.0, 47.6574, -122.278, 2200.0, 5000.0, 88.0, 0.0, 0.0]",[1026997.9],1,True
3948,2024-04-22 21:06:25.675,"[5.0, 4.25, 4720.0, 18741.0, 2.0, 0.0, 0.0, 3.0, 11.0, 3210.0, 1510.0, 47.6347, -122.013, 3880.0, 37328.0, 9.0, 0.0, 0.0]",[1151932.0],1,True
3957,2024-04-22 21:06:25.675,"[5.0, 3.5, 3620.0, 7821.0, 2.0, 0.0, 2.0, 3.0, 10.0, 2790.0, 830.0, 47.5738, -122.215, 2690.0, 9757.0, 56.0, 1.0, 52.0]",[1445844.0],1,True


## Clean Up

At this point, if you are not continuing on to the next notebook, undeploy your pipeline to give the resources back to the environment.

In [7]:
## blank space to undeploy the pipeline

pipeline.undeploy()

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-22 21:06:21.434281+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


## Congratulations!

In this workshop you have

* Set a validation rule on your house price prediction pipeline.
* Detected model predictions that failed the validation rule.

In the next notebook, you will learn how to monitor the distribution of model outputs for drift away from expected behavior.